# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [32]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
import datetime
import cv2
import matplotlib.pyplot as plt

GPU limitation

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-07-06 10:55:17.818344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.955122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.955372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.992199: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Model

### Train

In [6]:
default_data = np.loadtxt('defaultDiff.txt')
falling_data = np.loadtxt('fallingDiff.txt')

In [23]:
mu, sig = 0, 0.001
copy_count = len(default_data)-len(falling_data)
oldLen = len(falling_data)
rowi = 0

# limit = lambda x : max(min(x,1), 0)

def limit(x):
    if x > 1: return 1 - (x-1)
    elif x < 0: return abs(x)
    return x

newLimit = np.vectorize(limit)

for i in range(copy_count):
    # err = np.random.normal(mu, sig, 16)
    err = 0
    scale = 1
    row = falling_data[rowi]*scale + err
    row = newLimit(row)

    falling_data = np.vstack([falling_data, row])
    rowi += 1
    if rowi >= oldLen: rowi = 0

print(falling_data.shape, default_data.shape)

(3220, 16) (3220, 16)


In [29]:
# loading data
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default
dataset_size = len(inputs)
new_indices = np.random.permutation(dataset_size) # shuffle indices to shuffle X and y at the same time
inputs, outputs = inputs[new_indices], outputs[new_indices]

train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size

train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size]
X_test, y_test = inputs[train_size:], outputs[train_size:]
print(X_test)

[[0.09180733 0.09466914 0.10662924 ... 0.11981803 0.0934365  0.08545271]
 [0.03829182 0.03852374 0.04664954 ... 0.09508381 0.09578914 0.09532939]
 [0.06093381 0.05540801 0.06446455 ... 0.08758888 0.08465607 0.09008838]
 ...
 [0.34471962 0.33762761 0.22215499 ... 0.26457646 0.23173583 0.2225707 ]
 [0.05326341 0.02523982 0.0726422  ... 0.07243748 0.07156026 0.07870207]
 [0.01184504 0.01083142 0.01386972 ... 0.01039466 0.01052505 0.0111747 ]]


In [30]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [31]:
model.fit(x=X_train, y=y_train, validation_split=0.1, epochs=6,callbacks=[tensorboard_callback]) # if training data is np array
# model.fit(train_data, epochs=6,callbacks=[tensorboard_callback]) # if training data is tensor

Epoch 1/6
145/145 [==============================] - 1s 5ms/step - loss: 0.5831 - accuracy: 0.6592 - val_loss: 0.5423 - val_accuracy: 0.6938
Epoch 2/6
145/145 [==============================] - 0s 1ms/step - loss: 0.5151 - accuracy: 0.7332 - val_loss: 0.5379 - val_accuracy: 0.6996
Epoch 3/6
145/145 [==============================] - 0s 1ms/step - loss: 0.5028 - accuracy: 0.7414 - val_loss: 0.5298 - val_accuracy: 0.7016
Epoch 4/6
145/145 [==============================] - 0s 1ms/step - loss: 0.4977 - accuracy: 0.7433 - val_loss: 0.5296 - val_accuracy: 0.7093
Epoch 5/6
145/145 [==============================] - 1s 4ms/step - loss: 0.4934 - accuracy: 0.7476 - val_loss: 0.5232 - val_accuracy: 0.7151
Epoch 6/6
145/145 [==============================] - 0s 1ms/step - loss: 0.4897 - accuracy: 0.7500 - val_loss: 0.5184 - val_accuracy: 0.7132


### Plot Data

In [38]:
for r in range(len(falling_data)):
    x = np.arange(16)
    y = falling_data[r]
    plt.plot(x,y,'r',label="falling")
for r in range(len(default_data)):
    x = np.arange(16)
    y = default_data[r]
    plt.plot(x,y,'b',label="default")
plt.legend()
plt.show()


/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/IPython/core/pylabtools.py:152: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 547x134913 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>

### Evaluate & Export

In [30]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
# model.save("new_model2")

40/40 [==============================] - 0s 991us/step - loss: 0.3782 - accuracy: 0.8485


In [8]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('new_model2.tflite', 'wb') as f: f.write(model2_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpgy6fdty2/assets


2023-06-07 11:16:44.155686: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-06-07 11:16:44.445558: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 11:16:44.445580: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 11:16:44.446467: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.447205: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 11:16:44.447233: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.449735: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-06-07 11:16:44.489124: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on Saved

In [29]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('new_model2_quant', 'wb') as f: f.write(tflite_quant_model2)

INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets


INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-07 15:22:06.987031: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 15:22:06.987052: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 15:22:06.987225: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.987830: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 15:22:06.987841: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.989672: I tensorflow/cc/sav